In [1]:
def choose_target_features(target,feature,confirm):
   
    global df
    

    if  target == feature:
        
        print(c_bold,c_red,"\nTarget and feature can't be the same ! \n")
        ws.Beep(1000,1000)
        
    elif confirm == 'Yes': 
    
        build_model(df,target,feature)
        
    else:
        
        print('')
        
    return


def next_color(p_par): 
    
    global x_color
    
    if  p_par == 1:
  
        if  x_color == 36:
            x_color = 30

        x_color += 1       
        
    else:    

        x_color = 31 # Return to first color
       
                
    return('\033[1m \033[%im' % (x_color))


    
def build_model(df,target,feature):   
   
    
    global x_test,y_test
    
   
    if feature == '- All features -': 

        df_feature_corr_level = df.corr()[target][df.corr()[target] >= 0.7].sort_values()
        df_feature_corr_level.drop(labels = target, inplace = True)
        
        print(next_color(1),
              '\n Features identified on file that present a linear tendency with the target',
               next_color(1),
               target.upper(),
               next_color(0),
               '(correlation coeficient >= 0.7):') # goes back to first color

        print(next_color(1))

        for index,value in enumerate(df_feature_corr_level):
            print('  -  ' + df_feature_corr_level.index[index],' (%f' % (value)+')')      

        print('\n')


        combinations = list(itertools.chain(*[itertools.combinations(df_feature_corr_level.index, i+1) for i in range(len(df_feature_corr_level.index))]))
        
        
    else:    
        
        if  df.corr()[target][feature] < 0.7:
            comparison_to_07 = '(< 0.7)'
        else:    
            comparison_to_07 = '(>= 0.7)'


        print(c_bold+c_red
              +'\n Correlation coeficient between target',
              c_bold,c_blue,
              target.upper(),
              c_bold,c_red,
              'and feature',
              c_bold,c_blue,
              feature.upper(),
              c_bold,c_red,
              '=====> '
              +c_bold+c_yellow,
              df.corr()[target][feature],
              c_bold+c_red,
              comparison_to_07)

        combinations = [(feature,)]


    best_score = 0
    best_combination = ""
    best_number_of_parts = 0



    lr = LinearRegression()
    sd = StandardScaler()

    data_out = 0  # to be removed later (processing just 75% of data)

    
    
    if  data_out != 0:  # in case of precision test
  
        df,x_test,df[target],y_test = train_test_split(df.drop(target,axis=1),df[target],test_size=(data_out/100),random_state=0)
    
    
    print(next_color(1),'\n Using %i'  % (100-data_out),'%','percentage of data to build up model')
    #print(next_color(1),'\n Building up model...')

    for order in range(1,6,1):

        pf = PolynomialFeatures(degree=order,include_bias = None)    

        if len(combinations) > 1:
            print(next_color(1),'\n\n ------- Checking relevant features for an order-%i model...\n ' % (order))
        else:
            print(next_color(1),'\n\n ------- Checking scores for an order-%i model...\n ' % (order))
            

        for n_parts in range(2,6,1):

            if len(combinations) > 1:
                print('Grid-Searching them with %i fold(s) ...' % (n_parts))
            else:
                print('Grid-Searching it with %i fold(s) ...' % (n_parts))

                
            for features in combinations:

                gs = GridSearchCV(lr,{},cv=n_parts)

                poly_boxes = pf.fit_transform(df[list(features)])

                features_score = gs.fit(poly_boxes,df[target]).best_score_


                if  features_score >  best_score: 
                    best_score =  features_score
                    best_order = order
                    best_combination = list(features)
                    best_number_of_parts =  n_parts

 
        if  len(combinations) > 1:
            print('\n ----- Results ------> Most relevant futures sor far: ',best_combination)   
 
        if best_score > 0:
            print('\n                       Best order so far............: ',best_order)        
            print('\n                       Best cross-validation........: ',best_number_of_parts,' fold(s)')  
            print('\n                       Score for above..............: ',best_score)


    ##################### Evaluates any improvement of model above with Ridge Regression ##################        

    
    
    print(next_color(1),'\n\n Using Ridge Regression to improve model with the paramaters above ...')


    pf = PolynomialFeatures(degree=best_order,include_bias=False)

    poly_boxes = pf.fit_transform(sd.fit_transform(df[best_combination]))

    rr = Ridge()

    standard_amount = 10**-6

    alpha_max = 0

    increment = 0   


    while increment != standard_amount:

        increment = standard_amount

        print('\n Grid-Searching with alpha: ',alpha_max + increment)


        while True:

            gs = GridSearchCV(rr,{'alpha':[alpha_max + increment]},cv=best_number_of_parts)
            score_1 = gs.fit(poly_boxes,df[target]).best_score_

            gs = GridSearchCV(rr,{'alpha':[alpha_max + increment+standard_amount]},cv=best_number_of_parts)
            score_2 = gs.fit(poly_boxes,df[target]).best_score_


            if  score_1 <= score_2:
                increment *= 2  
            else:
                if  increment != standard_amount: # processed above at least once 
                    alpha_max += increment/2 # equal last increment 
                break


    best_alpha = alpha_max

    gs = GridSearchCV(rr,{'alpha':[alpha_max]},cv=best_number_of_parts)

    print('\n New score obtained: %f (%.2f' 
                           %
                           (gs.fit(poly_boxes,df[target]).best_score_, 
                           ((gs.fit(poly_boxes,df[target]).best_score_/best_score)-1)*100),
                           '% improvement) \n')   

    
    
    ########################## Model constituted by (1-1/cv) [Training Data] #################################
    ################## This part will show performance of model over unseen (1/cv) [Testing Data]  



    pf = PolynomialFeatures(degree=best_order,include_bias=False)

    poly_boxes = pf.fit_transform(sd.fit_transform(df[best_combination]))

    lr = Ridge(best_alpha)

    y_predict = cross_val_predict(lr,poly_boxes,df[target],cv=best_number_of_parts)

    plt.figure(figsize=(12,8))

    plt.title('Population Distribution on '+ target)
    plt.ylabel('Population')

    sns.distplot(df[target], hist=False, color='r', label='Actual '+ target)
    sns.distplot(y_predict,   hist=False, color='b', label='Predicted '+ target)

    plt.legend(loc='best')

    training_percentage = (1-(1/best_number_of_parts))*100

    print(next_color(1))

    print('\n********* FINAL RESULTS ***********\n')

    print('Polynomial order................:  %i' % (best_order))

    print('Bias included (alpha)...........:  %f' % (best_alpha))

    print('Predicted Price.................:  Model constituted on ' 
                         +'%.2f' % (training_percentage),
                          '%',
                          'of Training Data (Cross Validation with %i parts' % (best_number_of_parts)
                         +')')

    print('R2 on Testing Data %.2f' % (100-training_percentage),
                          '%','.....: ',r2_score(df[target],y_predict),
                          '(Cross Validation with %i parts' % (best_number_of_parts)+')')


    ################### This part will show model constituted by 100% Data ########                

    pf = PolynomialFeatures(degree=best_order,include_bias=False)

    poly_boxes = pf.fit_transform(df[best_combination])

    all_terms = pf.get_feature_names()

    for term_position,term in enumerate(all_terms):

        for feature_position,feature in enumerate(best_combination):
            all_terms[term_position]= all_terms[term_position].replace('x'+str(feature_position),"("+feature+")") 


    lr.fit(poly_boxes,df[target])
    
    
    if  len(best_combination) == 1:  #just 1 dimension
        
        plt.figure(figsize=(16,6))
        
        plt.title(best_combination[0] + ' X ' +  target)
        plt.xlabel(best_combination[0])
        plt.ylabel(target)
        
        plt.plot(df[best_combination],df[target],'ro',label='Data')
        
        poly_boxes_sorted = pf.fit_transform(pd.DataFrame(np.sort(df[best_combination[0]])))
       
        plt.plot(np.sort(df[best_combination[0]]), lr.predict(poly_boxes_sorted),'b',linewidth = "3.0",label='Model')
        
        plt.legend(loc='best')


    print(next_color(1))

    print('R2 on Training Data 100',   '%','......: ',lr.score(poly_boxes,df[target]))
    print('Model on Training Data 100','%','...:  yhat = %+f' % (lr.intercept_))
    for parameter in range(0,len(lr.coef_),1):
        print(' '*42+" %+f" % (lr.coef_[parameter]) + "*("+ all_terms[parameter] + ")")    

    print(color_off)


"""  ##############################################################################
    if  data_out != 0:  # in case of precision test
    
        poly_boxes = pf.fit_transform(x_test[best_combination])

        print(lr.score(poly_boxes,y_test))
        

        print(r2_score(y_test,lr.predict(poly_boxes)))
############################################################################## """     



    
############################# Cross Validation with Ridge Regression Refinement


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from ipywidgets import interact

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import winsound as ws
import pandas as pd
import numpy as np
import itertools




warnings.simplefilter('ignore')

x_color = 35

c_bold = '\033[1m' 
c_red = "\033[31m" 
c_blue = '\033[34m' 
c_yellow = '\033[33m'
color_off = '\033[0m'


print(next_color(1),'\n','Machine Learning - Linear Regression Model builder'.upper(),'\n')


while True:
    
    print(next_color(1),'\nInforme path/name of CSV file:  (<ENTER> to abort)')
    file_name = input()
        
    if  file_name == '':

        print('Program aborted !')
        print('---------------\n')
        ws.Beep(500,1000)
        break
        
    try:
            
        print('\nOpening file ...')
        df = pd.read_csv((file_name.strip()))
        print('Done !')

        #'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/module_5_auto.csv'

        df=df._get_numeric_data()

        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)

        options   = df.columns.sort_values()

        break

    except: 
 
        if  IOError:
            print('File not found !\n')
        else:
            print('Unidentified error !\n')
   
        ws.Beep(1000,1000)

if  file_name != '':

    interact(choose_target_features, target = options, feature=options.insert(0,'- All features -'), confirm=['No','Yes'])

    


  
 MACHINE LEARNING - LINEAR REGRESSION MODEL BUILDER 

  
Informe path/name of CSV file:  (<ENTER> to abort)
C:/Users/p_com/Downloads/auto_prices.csv

Opening file ...
Done !


interactive(children=(Dropdown(description='target', options=('Unnamed: 0', 'Unnamed: 0.1', 'bore', 'city-L/10…